In [1]:
import pickle
import pandas as pd
import re

In [2]:
df1 = pd.read_csv('QueryResults apt.csv')
df2 = pd.read_csv('QueryResults bottom 50.csv')
df3 = pd.read_csv('QueryResults command-line.csv')
df4 = pd.read_csv('QueryResults Scripts.csv')
df5 = pd.read_csv('QueryResults top 50.csv')
df6 = pd.read_csv('QueryResults bash.csv')

In [3]:
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

In [4]:
len(df)

72351

In [5]:
df.drop_duplicates(subset='Id', keep='first', ignore_index=True, inplace=True)

In [6]:
len(df)

57464

In [7]:
df.shape

(57464, 12)

In [8]:
df

,Post Link,TagName,Title,AnswerCount,FavoriteCount,AcceptedAnswerId,PostTypeId,Id,CreationDate,DeletionDate,Score,ViewCount
0,https://askubuntu.com/questions/307,apt,How can PPAs be removed?,26,459.0,310.0,1,307,29-07-2010 09:09,NaN,1361,1559164
1,https://askubuntu.com/questions/365,apt,Is it possible to tell what packages I've inst...,4,12.0,371.0,1,365,29-07-2010 13:06,NaN,24,1974
2,https://askubuntu.com/questions/519,apt,How do I write an application install shell sc...,6,26.0,520.0,1,519,30-07-2010 18:30,NaN,43,118801
3,https://askubuntu.com/questions/529,apt,How to set up an APT repository?,5,34.0,532.0,1,529,31-07-2010 00:33,NaN,54,66360
4,https://askubuntu.com/questions/737,apt,Authentication problem with my PPA,1,1.0,739.0,1,737,03-08-2010 20:07,NaN,11,1253
...,...,...,...,...,...,...,...,...,...,...,...,...
57459,https://askubuntu.com/questions/1289599,bash,SNAP leaves behind some references after remov...,1,NaN,1289951.0,1,1289599,2020-11-04 15:44:14,NaN,0,23
57460,https://askubuntu.com/questions/1289618,bash,Bash script for Cisco Anyconnect VPN (error pa...,1,NaN,1289619.0,1,1289618,2020-11-04 17:04:24,NaN,0,21
57461,https://askubuntu.com/questions/1289859,bash,bash --> perl command: print only the replaced...,2,NaN,1289868.0,1,1289859,2020-11-05 14:04:15,NaN,2,102
57462,https://askubuntu.com/questions/1290381,bash,How to remove low abundance and less prevalent...,0,NaN,NaN,1,1290381,2020-11-07 13:30:24,NaN,0,16


In [9]:
id_to_index = {df.iloc[i, 7]:i for i in range(len(df))}

In [10]:
li1 = pickle.load(open('bash_file_codes_0_1000.txt', 'rb'))
li2 = pickle.load(open('bash_file_codes_1000_3000.txt', 'rb'))
li3 = pickle.load(open('bash_file_codes_3000_4112.txt', 'rb'))
li4 = pickle.load(open('bash_file_codes_4112_6000.txt', 'rb'))
li5 = pickle.load(open('bash_file_codes_6000_9000.txt', 'rb'))
li6 = pickle.load(open('bash_file_codes_9000_10000.txt', 'rb'))

In [11]:
print(len(li1), len(li2), len(li3), len(li4), len(li5), len(li6))

1000 1999 1112 1887 2998 998


In [12]:
for j in [li2, li3, li4, li5, li6]:
    li1.extend(j)

list_10k = li1

In [13]:
len(list_10k)

9994

In [14]:
#A LOOK AT THE NOISY DATA

list_10k[0:2]

[{'Id': 307,
  'Question Body': "\nI've added many PPAs using the add-apt-repository command. Is there a simple way to remove these PPAs? I've checked in /etc/apt/sources.list for the appropriate deb lines but they aren't there. \nThis is on a server system so a command line solution would be great!\n",
  'Answers': {'Accepted Answer': {'Answer Id': 'answer-310',
    'Answer Text': '\n\n\n\n\n1706\n\n\n\n\n\n\n\n\n\n\n\nThere are a number of options:\n\nUse the --remove flag, similar to how the PPA was added:\nsudo add-apt-repository --remove ppa:whatever/ppa\n\n\nYou can also remove PPAs by deleting the .list files from /etc/apt/sources.list.d directory.\n\nAs a safer alternative, you can install ppa-purge:\nsudo apt-get install ppa-purge\n\nAnd then remove the PPA, downgrading gracefully packages it provided to packages provided by official repositories:\nsudo ppa-purge ppa:whatever/ppa\n\nNote that this will uninstall packages provided by the PPA, but not those provided by the offic

In [15]:
#FOR NOW FILTERING OUT ONES WITHOUT ACCEPTED ANSWERS, LATER MIGHT EXTEND

list_with_accepted_answers = []

for i in range(len(list_10k)):
    if('Accepted Answer' in list_10k[i]['Answers'].keys()):
        list_with_accepted_answers.append(list_10k[i]) 

In [16]:
#NUMBER OF POSTS WITH ACCEPTED ANSWERS

len(list_with_accepted_answers)

5887

In [17]:
#TAKING POSTS WITH CODES IN ANSWER 1

list_with_codes_in1 = []
for i in range(len(list_with_accepted_answers)):
    if(len(list_with_accepted_answers[i]['Answers']['Answer1']['Codes'])!=0):
        list_with_codes_in1.append(list_with_accepted_answers[i])


In [18]:
#TAKING POSTS WITH CODES IN EITHER ANSWER 1 OR ANSWER 2

list_with_codes_in2 = []
for i in range(len(list_with_accepted_answers)):
    if(len(list_with_accepted_answers[i]['Answers']['Answer1']['Codes'])!=0):
        list_with_codes_in2.append(list_with_accepted_answers[i])
    elif('Answer2' in list_with_accepted_answers[i]['Answers'].keys()):
        if(len(list_with_accepted_answers[i]['Answers']['Answer2']['Codes'])!=0):
            list_with_codes_in2.append(list_with_accepted_answers[i])  


In [19]:
len(list_with_codes_in1)

4694

In [20]:
len(list_with_codes_in2)

5030

In [21]:
#QUESTIONS WHICH ARE NOT CLOSED


list_not_closed = []

for i in range(len(list_with_codes_in2)):
    if(len(re.findall(r'closed \d years ago', list_with_codes_in2[i]['Question Body'].lower()))==0):
        list_not_closed.append(list_with_codes_in2[i])       
        

In [22]:
len(list_not_closed)

4740

In [23]:
#REMOVING THOSE WITH ONLY ONE WORD CODE

list_real_codes = []

for i in range(len(list_not_closed)):
    f=0
    for j in range(len(list_not_closed[i]['Answers']['Answer1']['Codes'])):
        if(len(list_not_closed[i]['Answers']['Answer1']['Codes'][j].split(' '))>1):
            f=1
            break
    
    if('Answer2' in list_not_closed[i]['Answers'].keys()):
        for j in range(len(list_not_closed[i]['Answers']['Answer2']['Codes'])):        
            if(len(list_not_closed[i]['Answers']['Answer2']['Codes'][j].split(' '))>1):
                f=1
                break
    if(f==1):
        list_real_codes.append(list_not_closed[i])
        

In [24]:
len(list_real_codes)

4292

In [25]:
#REMOVING ONE WORD CODE ELEMENTS
for i in range(len(list_real_codes)):
    l = len(list_real_codes[i]['Answers']['Answer1']['Codes'])
    nl = []
    for j in range(l):
        if(len(list_real_codes[i]['Answers']['Answer1']['Codes'][j].split(' '))>1):
            nl.append(list_real_codes[i]['Answers']['Answer1']['Codes'][j])
    list_real_codes[i]['Answers']['Answer1']['Codes'] = nl
    list_real_codes[i]['Answers']['Accepted Answer']['Codes'] = nl
    
    if('Answer2' in list_real_codes[i]['Answers'].keys()):
        l = len(list_real_codes[i]['Answers']['Answer1']['Codes'])
        nl = []
        for j in range(l):
            if(len(list_real_codes[i]['Answers']['Answer1']['Codes'][j].split(' '))>1):
                nl.append(list_real_codes[i]['Answers']['Answer1']['Codes'][j])
        list_real_codes[i]['Answers']['Answer1']['Codes'] = nl
        list_real_codes[i]['Answers']['Accepted Answer']['Codes'] = nl

In [26]:
#NUMBER OF POSTS WITH A STANDALONE ANSWER

count = 0
for i in range(len(list_real_codes)):
    l1 = len(list_real_codes[i]['Answers']['Answer1']['Codes'])
   
    if('Answer2' in list_real_codes[i]['Answers'].keys()):
        l2 = len(list_real_codes[i]['Answers']['Answer2']['Codes'])
    
    if(l1==1 or l2==1):
        count=count+1
        
count

2296

In [27]:
# STANDALONE POSTS

standalone_posts=[]

for i in range(len(list_real_codes)):
    l1 = len(list_real_codes[i]['Answers']['Answer1']['Codes'])
   
    if('Answer2' in list_real_codes[i]['Answers'].keys()):
        l2 = len(list_real_codes[i]['Answers']['Answer2']['Codes'])
    
    if(l1==1 or l2==1):
        standalone_posts.append(list_real_codes[i])
        
len(standalone_posts)

2296

In [30]:
ind = id_to_index[standalone_posts[3]['Id']]

print(standalone_posts[3]['Id'])
print(df.iloc[ind, 2])

print(standalone_posts[3]['Answers']['Answer1']['Codes'][0])

2389
Generating list of manually installed packages and querying individual packages
comm -23 <(apt-mark showmanual | sort -u) <(gzip -dc /var/log/installer/initial-status.gz | sed -n 's/^Package: //p' | sort -u)

